# Setup

In [1]:
import pandas as pd
import numpy as np

import re
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx
from tqdm import tqdm

In [3]:
file_name = "/kaggle/input/inputfile/i_hc.csv"
df = pd.read_csv(file_name, encoding="utf-8-sig")

In [4]:
print(df)

        idx                                            content  \
0      -604  thông_báo trường đại_học cmc  ,  tân sinh_viên...   
1      -617  triển_khai kế_hoạch đào_tạo 3 năm lớp cao_đẳng...   
2      -983  học_phí năm_học 2024 2025 trường đại_học xây_d...   
3     -1065  cung không cầu anh đặng_duy_vũ xã hương_phong ...   
4     -1066  15/10  ,  đồng_chí trần_lê_đoài  ,  tuv  ,  ph...   
...     ...                                                ...   
1684  39941   ,  câu đại_diện việt_nam phát_triển toàn_cầu ...   
1685  40385  lễ tôn_vinh sinh_viên thành_tích xuất_sắc học_...   
1686  41701  tân_thủ_lĩnh thi miss_international 2024  ,  h...   
1687  43177  huỳnh_thị_thanh_thuỷ  ,  sinh năm 2002 đà_nẵng...   
1688  43212  clip 27/11  ,  hoa_hậu quốc_tế 2024 huỳnh_thị_...   

                                                   tags Chủ đề chính  
0     Đại học CMC,tân sinh viên,CMC,máy tính xách ta...      Đại học  
1     Cao đẳng Gia Lai,bằng tốt nghiệp,bế giảng,trun...      Đại 

## Hàm xóa stop words, số, loại bỏ từ đơn

In [5]:
with open('/kaggle/input/inputfile/StopWord.txt', 'r', encoding='utf-8') as f:
  vietnamese_stopwords = [word.strip() for word in f.readlines() if word.strip()]

In [6]:
def preprocess(text):
    text = str(text)
    
    # Giữ lại các dấu chấm, phẩy hợp lệ
    pattern_keep = r'(?<=[a-zA-Z0-9])(\.|,)(?=[a-zA-Z0-9])|(?<=[a-zA-Z])\.(?=\s|,|$)'
    text = re.sub(pattern_keep, r'\1', text)
    
    # Loại bỏ các dấu chấm, phẩy không mong muốn
    pattern_remove = r'[.,](?=\s|$|[^a-zA-Z0-9])'
    cleaned_text = re.sub(pattern_remove, ' ', text)

    # Loại bỏ các khoảng trắng dư thừa và các ký tự không mong muốn
    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text)
    cleaned_text = re.sub(r'[/-]', ' ', cleaned_text)
    cleaned_text = re.sub(r'\b\d+\b', '', cleaned_text)
    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text)

    # Loại bỏ địa chỉ IP và dấu chấm dư thừa
    pattern_ip = r'\b(?:\d{1,3}\.){3}\d{1,3}\b'
    cleaned_text = re.sub(pattern_ip, '', cleaned_text)
    pattern_remove_dots = r'\.'
    cleaned_text = re.sub(pattern_remove_dots, '', cleaned_text)

    # Định dạng lại dấu chấm và phẩy
    cleaned_text = cleaned_text.replace(',', ' , ').replace('.', ' . ')
    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text)

    words = cleaned_text.split()
    processed_words = [' , ' if '_' not in word and word not in {',', '.'} else word for word in words]

    return ' '.join(processed_words).strip()

# Chạy mô hình trích xuất từ khóa

---



In [7]:
text_column = "content"

texts_in_cluster = df[text_column].dropna().tolist()
texts_in_cluster = [preprocess(text) for text in texts_in_cluster]

# Bước 1: Chuẩn bị dữ liệu huấn luyện Word2Vec
tokenized_texts = [text.split() for text in texts_in_cluster]

# Bước 2: Huấn luyện Word2Vec
word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=50, window=2, min_count=1, workers=4, epochs=50)

# Bước 3: Trích xuất từ vựng
# Sử dụng CountVectorizer để tìm tập từ phổ biến
vectorizer = CountVectorizer(max_features=25)
vectorizer.fit(texts_in_cluster)
cluster_vocab = set(vectorizer.get_feature_names_out())

# In từ vựng
print(f"Tập từ vựng chung từ Word2Vec (tối đa 25 từ): {cluster_vocab}")

Tập từ vựng chung từ Word2Vec (tối đa 25 từ): {'quốc_tế', 'tổ_chức', 'đào_tạo', 'công_nghệ', 'hà_nội', 'chương_trình', 'học_tập', 'việt_nam', 'kỹ_năng', 'giáo_dục', 'nghiên_cứu', 'phát_triển', 'kiến_thức', 'tốt_nghiệp', 'học_bổng', 'sinh_viên', 'học_sinh', 'tham_gia', 'quốc_gia', 'chất_lượng', 'nhà_trường', 'thí_sinh', 'đại_học', 'hoạt_động', 'khoa_học'}


In [8]:
# Hàm kiểm tra ngưỡng đủ tốt dựa trên Word2Vec
def filter_keywords_by_similarity(doc_keywords, cluster_vocab, model, threshold=0.5):
    filtered_keywords = []
    for keyword in doc_keywords:
        if keyword in model.wv:  # Chỉ xét các từ có trong Word2Vec
            max_similarity = max(
                cosine_similarity([model.wv[keyword]], [model.wv[vocab_word]])[0][0]
                for vocab_word in cluster_vocab if vocab_word in model.wv
            )
            if max_similarity >= threshold:
                filtered_keywords.append(keyword)
    return filtered_keywords

# Bước 4: Áp dụng HITS với cải tiến
results = []
indices = df["idx"]

for idx, text in tqdm(zip(indices, texts_in_cluster), desc="Processing Texts", total=len(texts_in_cluster)):
    if idx < 0:
        continue

    text = preprocess(text)
    words = text.split()
    word_count = len(words)

    graph = nx.DiGraph()  # Đồ thị có hướng

    window_size = 3
    for i in range(word_count):
        for j in range(i + 1, min(i + window_size, word_count)):
            graph.add_edge(words[i], words[j])  # Cạnh có hướng từ từ i -> từ j

    try:
        # Tính điểm HITS
        hubs, authorities = nx.hits(graph, max_iter=100, normalized=True)

        # Chọn các từ khóa dựa trên điểm authority cao nhất
        doc_keywords = sorted(authorities, key=authorities.get, reverse=True)[:int(0.2 * len(authorities))]

        # Lọc các từ khóa dựa trên ngưỡng Word2Vec
        doc_keywords = filter_keywords_by_similarity(doc_keywords, cluster_vocab, word2vec_model, threshold=0.5)
    except Exception as e:
        print(f"Error processing text: {e}")
        doc_keywords = []

    # Thêm các từ trong cluster_vocab nếu chưa có trong doc_keywords
    for word in cluster_vocab:
        if word in text and word not in doc_keywords:
            doc_keywords.append(word)

    results.append({
        'idx': idx,
        'text': text,
        'tags': ', '.join(doc_keywords)
    })

result_df = pd.DataFrame(results)
print(result_df.head())

Processing Texts: 100%|██████████| 1689/1689 [00:06<00:00, 257.78it/s] 

    idx                                               text  \
0   138  sinh_viên đại_học xây_dựng trung_muce tổ_chức ...   
1  2327  nguyễn_tấn_hoà giám_đốc trung_tâm sinh_viên qu...   
2  2604  nổi_bật lĩnh_vực giáo_dục hoài_đức hoài_đức ph...   
3  6599  thiếu_tướng nguyễn_ngọc_thanh tư_lệnh bộ_tư_lệ...   
4  8053  khai_giảng giám_đốc đại_học bách_khoa hà_nội g...   

                                                tags  
0  sinh_viên, đại_học, hoạt_động, tham_gia, thuận...  
1  sinh_viên, đại_học, doanh_nghiệp, việc_làm, là...  
2  học_sinh, phát_triển, quý_báu, hà_nội, tham_gi...  
3  học_viên, chương_trình, đại_học, văn_bằng, tổ_...  
4  đại_học, bách_khoa, sinh_viên, phát_triển, hiệ...  


In [9]:
print(result_df.tail())

      idx                                               text  \
72  39941  đại_diện việt_nam phát_triển toàn_cầu tác_động...   
73  40385  tôn_vinh sinh_viên thành_tích xuất_sắc học_kỳ ...   
74  41701  tân_thủ_lĩnh miss_international huỳnh_thị_than...   
75  43177  huỳnh_thị_thanh_thuỷ đà_nẵng sở_hữu hoa_hậu qu...   
76  43212  hoa_hậu quốc_tế huỳnh_thị_thanh_thuỷ giao_lưu ...   

                                                 tags  
72  phát_triển, quốc_tế, tổ_chức, công_nghệ, học_t...  
73  đại_học, việt_nam, quốc_tế, xuất_sắc, giáo_dục...  
74  đại_học, việt_nam, đà_nẵng, khả_năng, chung_kế...  
75  đại_học, việt_nam, đà_nẵng, sinh_viên, quốc_tế...  
76  đà_nẵng, việt_nam, quốc_tế, tổ_chức, học_tập, ...  


In [10]:
output_file = "/kaggle/working/Dai_hoc.csv"
result_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"Kết quả được lưu tại: {output_file}")

Kết quả được lưu tại: /kaggle/working/Dai_hoc.csv


In [11]:
print(result_df.head())

    idx                                               text  \
0   138  sinh_viên đại_học xây_dựng trung_muce tổ_chức ...   
1  2327  nguyễn_tấn_hoà giám_đốc trung_tâm sinh_viên qu...   
2  2604  nổi_bật lĩnh_vực giáo_dục hoài_đức hoài_đức ph...   
3  6599  thiếu_tướng nguyễn_ngọc_thanh tư_lệnh bộ_tư_lệ...   
4  8053  khai_giảng giám_đốc đại_học bách_khoa hà_nội g...   

                                                tags  
0  sinh_viên, đại_học, hoạt_động, tham_gia, thuận...  
1  sinh_viên, đại_học, doanh_nghiệp, việc_làm, là...  
2  học_sinh, phát_triển, quý_báu, hà_nội, tham_gi...  
3  học_viên, chương_trình, đại_học, văn_bằng, tổ_...  
4  đại_học, bách_khoa, sinh_viên, phát_triển, hiệ...  


## Xem xét những từ khóa này có thuộc Tags chuẩn không

In [12]:
clean_vocab = {word.replace("_", " ") for word in cluster_vocab}
tags = df["tags"].dropna().str.split(",").explode().str.strip()

unique_tags = set(tags)

matching_words = clean_vocab & unique_tags

overlap_percentage = len(matching_words) / len(clean_vocab) * 100

print(f"Số từ trùng lặp: {len(matching_words)}")
print(f"Tỷ lệ % trùng lặp: {overlap_percentage:.2f}%")

Số từ trùng lặp: 18
Tỷ lệ % trùng lặp: 72.00%


In [13]:
clean_vocab = {word.replace("_", " ") for word in cluster_vocab}

tags = df["tags"].dropna().str.split(",").explode().str.strip()

word_counts = {word: tags.str.contains(fr'\b{word}\b', regex=True).sum() for word in clean_vocab}

total_count = sum(word_counts.values())

total_tags = len(tags)
ratio = total_count / total_tags * 100

print("Số lần xuất hiện của từng từ trong clean_vocab:")
for word, count in word_counts.items():
    print(f"'{word}': {count}")

print(f"Tổng số lần các từ trong clean_vocab xuất hiện: {total_count}")
print(f"Tỷ lệ so với tổng số tags: {ratio:.2f}%")

Số lần xuất hiện của từng từ trong clean_vocab:
'quốc tế': 226
'chương trình': 142
'quốc gia': 36
'công nghệ': 78
'việt nam': 0
'nghiên cứu': 92
'nhà trường': 77
'chất lượng': 43
'khoa học': 180
'tham gia': 8
'giáo dục': 138
'học tập': 147
'kỹ năng': 99
'thí sinh': 104
'học sinh': 373
'kiến thức': 26
'tốt nghiệp': 222
'phát triển': 21
'hoạt động': 4
'hà nội': 2
'đại học': 445
'sinh viên': 670
'đào tạo': 235
'tổ chức': 28
'học bổng': 416
Tổng số lần các từ trong clean_vocab xuất hiện: 3812
Tỷ lệ so với tổng số tags: 11.85%


# Chuẩn hoá

In [15]:
try:
    if "tags" in result_df.columns:
        result_df["tags"] = result_df["tags"].str.replace("_", " ")
    else:
        print("The column 'tags' does not exist in the DataFrame.")
except Exception as e:
    print(f"Error: {e}")

result_df["tags"] = result_df["tags"].str.lower()
df["tags"] = df["tags"].str.lower()

In [16]:
print(result_df[["tags"]].head())
print(df["tags"].head())

                                                tags
0  sinh viên, đại học, hoạt động, tham gia, thuận...
1  sinh viên, đại học, doanh nghiệp, việc làm, là...
2  học sinh, phát triển, quý báu, hà nội, tham gi...
3  học viên, chương trình, đại học, văn bằng, tổ ...
4  đại học, bách khoa, sinh viên, phát triển, hiệ...
0    đại học cmc,tân sinh viên,cmc,máy tính xách ta...
1    cao đẳng gia lai,bằng tốt nghiệp,bế giảng,trun...
2    đại học xây dựng hà nội,học phí,cde,kde,mne,cd...
3    đhh,hướng nghiệp,đại học nông lâm,nông nghiệp,...
4    bệnh viện charite,trần lê đoài,đại học điều dư...
Name: tags, dtype: object
